# Equal-Weight NIFTY 500 Index Fund

## Introduction & Library Imports

The goal of this projrct is to create a Python script that will accept the value of your portfolio and tell you how many shares of each NIFTY 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this project.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the NIFTY 500.

The list of NIFTY 500 constituents is regularly updated on NSE India's site. [Click this link to download them now](https://archives.nseindia.com/content/indices/ind_nifty500list.csv). Now we can import our stocks list here.

In [2]:
stocks = pd.read_csv('ind_nifty500list.csv')
stocks

,Symbol
0,3MINDIA
1,ABB
2,POWERINDIA
3,ACC
4,AIAENG
...,...
496,YESBANK
497,ZEEL
498,ZENSARTECH
499,ZYDUSWELL


## Acquiring real time data from the market

Real time data for the market can be requested using API integration, which are often paid. However here I have used a free tool `nsetools` for our purpose. 

To install nsetools --> `pip install nsetools`

nsetools documentation - [Click here](https://nsetools.readthedocs.io/en/latest/index.html) 

In [17]:
from secrets import ALPHA_API_TOKEN
from nsetools import Nse
nse = Nse()

## Making Our First Request

We need to import only the stock price for this project. Therefore we will use nsetools to get 

* Price of each stock



In [21]:
symbol = '3MINDIA'
data = nse.get_quote(symbol)
data

{'pricebandupper': 26163.2,
 'symbol': '3MINDIA',
 'applicableMargin': 17.19,
 'bcEndDate': '26-AUG-20',
 'totalSellQuantity': None,
 'adhocMargin': None,
 'companyName': '3M India Limited',
 'marketType': 'N',
 'exDate': '19-AUG-20',
 'bcStartDate': '21-AUG-20',
 'css_status_desc': 'Listed',
 'dayHigh': 21900.0,
 'basePrice': 21802.7,
 'securityVar': 13.69,
 'pricebandlower': 17442.2,
 'sellQuantity5': None,
 'sellQuantity4': None,
 'sellQuantity3': None,
 'cm_adj_high_dt': '05-MAR-20',
 'sellQuantity2': None,
 'dayLow': 21481.0,
 'sellQuantity1': None,
 'quantityTraded': 3462.0,
 'pChange': '-0.04',
 'totalTradedValue': 752.39,
 'deliveryToTradedQuantity': 52.02,
 'totalBuyQuantity': None,
 'averagePrice': 21732.93,
 'indexVar': None,
 'cm_ffm': 6113.37,
 'purpose': 'ANNUAL GENERAL MEETING',
 'buyPrice2': None,
 'secDate': '19-Feb-2021 00:00:00',
 'buyPrice1': None,
 'high52': 24349.9,
 'previousClose': 21802.7,
 'ndEndDate': None,
 'low52': 15700.0,
 'buyPrice4': None,
 'buyPrice3':

## Parsing Our Call

The call that we executed in the last code block contains all of the information required to build our equal-weight NIFTY 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [22]:
price = data['lastPrice']
print(price)

21794.0


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. 

In [23]:
my_columns = ['Ticker', 'Stock Price', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Number of Shares to Buy


In [26]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        'N/A'
    ],index = my_columns 
    ),ignore_index=True
)

,Ticker,Stock Price,Number of Shares to Buy
0,3MINDIA,21794.0,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all NIFTY 500 stocks and store their data in the DataFrame using a `for` loop. This can take some time for 500 stocks.

In [30]:
final_dataframe = pd.DataFrame(columns=my_columns)
for stock in stocks['Symbol']:
    data = nse.get_quote(stock)
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data['lastPrice'],
        'N/A'
    ],index = my_columns 
    ),ignore_index=True
)

In [31]:
final_dataframe

,Ticker,Stock Price,Number of Shares to Buy
0,3MINDIA,21794.00,N/A
1,ABB,1518.00,N/A
2,POWERINDIA,1438.00,N/A
3,ACC,1749.95,N/A
4,AIAENG,1868.75,N/A
...,...,...,...
496,YESBANK,16.05,N/A
497,ZEEL,208.65,N/A
498,ZENSARTECH,251.00,N/A
499,ZYDUSWELL,1888.00,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

For an equal weight index, equal capital is invested in each stock. Also, we will round floating points to the smaller nearest integer.

In [39]:
portfolio_size = input('Enter the size of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print('Portfolio size should be numeric')
    portfolio_size = input('Enter the size of your portfolio:')
    val=float(portfolio_size)
    
#assumption has been made that user will input numeric value the second time.

Enter the size of your portfolio:1000000


In [40]:
position_size = val/len(final_dataframe.index)
#print(position_size)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i,'Number of Shares to Buy']= math.floor(position_size/final_dataframe.loc[i,'Stock Price'])
final_dataframe

,Ticker,Stock Price,Number of Shares to Buy
0,3MINDIA,21794.00,0
1,ABB,1518.00,1
2,POWERINDIA,1438.00,1
3,ACC,1749.95,1
4,AIAENG,1868.75,1
...,...,...,...
496,YESBANK,16.05,124
497,ZEEL,208.65,9
498,ZENSARTECH,251.00,7
499,ZYDUSWELL,1888.00,1


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files. 

### Initializing our XlsxWriter Object

In [71]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `₹`. We'll need three main formats for our Excel document:
* String format for tickers
* ₹XX.XX format for stock prices
* Integer format for the number of shares to purchase

In [72]:
bg_colour = '#0a0a23'
font_colour = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color' : font_colour,
        'bg_color' : bg_colour,
        'border' : 1
    }
)

rupee_format = writer.book.add_format(
    {
        'num_format' : '₹0.00',
        'font_color' : font_colour,
        'bg_color' : bg_colour,
        'border' : 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format' : '0',
        'font_color' : font_colour,
        'bg_color' : bg_colour,
        'border' : 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

Let's simplify this by putting it in 2 loops:

In [74]:
column_formats = {
    'A' : ['Symbol', string_format],
    'B' : ['Stock Price', rupee_format],
    'C' : ['Number of Shares to Buy', integer_format]
}
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1]) #this is for the column headers

## Saving Our Excel Output

Saving our Excel file is very easy:

In [75]:
writer.save()

This `.xlsx` file is shared with colleagues who make the necessary investments, giving us the position in Equal Weight NIFTY 500 index.